In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Substance_Use_Disorder"
cohort = "GSE125681"

# Input paths
in_trait_dir = "../../input/GEO/Substance_Use_Disorder"
in_cohort_dir = "../../input/GEO/Substance_Use_Disorder/GSE125681"

# Output paths
out_data_file = "../../output/preprocess/Substance_Use_Disorder/GSE125681.csv"
out_gene_data_file = "../../output/preprocess/Substance_Use_Disorder/gene_data/GSE125681.csv"
out_clinical_data_file = "../../output/preprocess/Substance_Use_Disorder/clinical_data/GSE125681.csv"
json_path = "../../output/preprocess/Substance_Use_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Brain Gene Expression Pattern of Individuals with Dual Diagnosis who died by suicide"
!Series_summary	"We compared the gene expression profile in the dorsolateral prefrontal area of subjects with dual diagnosis who died by suicide to the transcriptome of subjects with mood disorders and subjects with substance use disorder who died by suicide."
!Series_overall_design	"38 samples were collected."
Sample Characteristics Dictionary:
{0: ['age: 16', 'age: 17', 'age: 35', 'age: 26', 'age: 33', 'age: 19', 'age: 29', 'age: 18', 'age: 32', 'age: 23', 'age: 34', 'age: 39', 'age: 45', 'age: 58', 'age: 76', 'age: 55', 'age: 31', 'age: 24', 'age: 61', 'age: 90', 'age: 30', 'age: 53', 'age: 22', 'age: 27'], 1: ['gender: Male', 'gender: Female'], 2: ['suicide: Suicide'], 3: ['sud: SUD', 'sud: Without SUD']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the title and description, this likely contains gene expression data from the dorsolateral prefrontal area
is_gene_available = True  

# 2.1 Data Availability
# From the sample characteristics, we can identify keys for trait, age, and gender
trait_row = 3  # 'sud' (substance use disorder)
age_row = 0    # 'age'
gender_row = 1  # 'gender'

# 2.2 Data Type Conversion
def convert_trait(value):
    if not isinstance(value, str):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for SUD, 0 for Without SUD
    if value.lower() == 'sud':
        return 1
    elif value.lower() == 'without sud':
        return 0
    else:
        return None

def convert_age(value):
    if not isinstance(value, str):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to continuous numeric value
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    if not isinstance(value, str):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 0 for Female, 1 for Male
    if value.lower() == 'male':
        return 1
    elif value.lower() == 'female':
        return 0
    else:
        return None

# 3. Save Metadata
# Since trait_row is not None, trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features
# Since trait_row is not None, we proceed with feature extraction
if trait_row is not None:
    # Assume clinical_data is defined from a previous step
    try:
        selected_clinical_df = geo_select_clinical_features(
            clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")


Preview of selected clinical features:
{'GSM3580113': [1.0, 16.0, 1.0], 'GSM3580114': [1.0, 17.0, 1.0], 'GSM3580115': [1.0, 16.0, 1.0], 'GSM3580117': [1.0, 35.0, 1.0], 'GSM3580120': [1.0, 26.0, 1.0], 'GSM3580121': [1.0, 33.0, 1.0], 'GSM3580122': [1.0, 19.0, 1.0], 'GSM3580124': [1.0, 29.0, 1.0], 'GSM3580126': [0.0, 18.0, 0.0], 'GSM3580128': [0.0, 16.0, 0.0], 'GSM3580129': [0.0, 32.0, 1.0], 'GSM3580130': [0.0, 35.0, 1.0], 'GSM3580133': [0.0, 23.0, 1.0], 'GSM3580135': [0.0, 34.0, 1.0], 'GSM3580136': [1.0, 39.0, 1.0], 'GSM3580137': [0.0, 23.0, 0.0], 'GSM3580139': [1.0, 45.0, 1.0], 'GSM3580140': [0.0, 35.0, 1.0], 'GSM3580141': [0.0, 58.0, 1.0], 'GSM3580143': [0.0, 76.0, 1.0], 'GSM3580144': [1.0, 55.0, 1.0], 'GSM3580145': [0.0, 31.0, 1.0], 'GSM3580146': [0.0, 24.0, 0.0], 'GSM3580148': [1.0, 23.0, 1.0], 'GSM3580149': [1.0, 61.0, 1.0], 'GSM3580150': [1.0, 90.0, 1.0], 'GSM3580151': [1.0, 17.0, 1.0], 'GSM3580153': [1.0, 23.0, 0.0], 'GSM3580154': [1.0, 30.0, 1.0], 'GSM3580158': [0.0, 34.0, 1.0], 

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 62
Header line: "ID_REF"	"GSM3580113"	"GSM3580114"	"GSM3580115"	"GSM3580117"	"GSM3580120"	"GSM3580121"	"GSM3580122"	"GSM3580124"	"GSM3580126"	"GSM3580128"	"GSM3580129"	"GSM3580130"	"GSM3580133"	"GSM3580135"	"GSM3580136"	"GSM3580137"	"GSM3580139"	"GSM3580140"	"GSM3580141"	"GSM3580143"	"GSM3580144"	"GSM3580145"	"GSM3580146"	"GSM3580148"	"GSM3580149"	"GSM3580150"	"GSM3580151"	"GSM3580153"	"GSM3580154"	"GSM3580158"	"GSM3580164"	"GSM3580167"	"GSM3580168"	"GSM3580169"	"GSM3580171"	"GSM3580172"	"GSM3580174"	"GSM3580175"
First data line: "ILMN_1651229"	9.284170392	9.2246201	9.234463414	9.044533263	9.237307192	9.182889823	9.208731283	9.231173219	9.032337123	9.410774012	9.120015038	9.158365079	9.322335583	9.101266675	9.288959569	9.21875343	8.984921045	9.064043363	9.284498916	9.168211134	9.512112115	9.097280178	9.170814536	9.2760438	9.193590717	9.401714598	9.285201527	9.496871234	9.315469914	9.102376361	9.081554331	9.166868296	9.062749379	9.39293042	9.127197288	9.2414551

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# The identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# These are not standard human gene symbols and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE125681
Line 6: !Series_title = Brain Gene Expression Pattern of Individuals with Dual Diagnosis who died by suicide
Line 7: !Series_geo_accession = GSE125681
Line 8: !Series_status = Public on Dec 02 2019
Line 9: !Series_submission_date = Jan 25 2019
Line 10: !Series_last_update_date = Dec 02 2019
Line 11: !Series_pubmed_id = 31774731
Line 12: !Series_summary = We compared the gene expression profile in the dorsolateral prefrontal area of subjects with dual diagnosis who died by suicide to the transcriptome of subjects with mood disorders and subjects with substance use disorder who died by suicide.
Line 13: !Series_overall_design = 38 samples were collected.
Line 14: !Series_type = Expression profiling 


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# Extract the gene identifier and gene symbol columns from the gene annotation dataframe
# Based on the preview, the 'ID' column in gene_annotation matches the probe IDs in gene_data (both start with ILMN_)
# The 'Symbol' column contains the gene symbols we need to map to

try:
    # Create the mapping dataframe with the ID and Symbol columns
    mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
    
    # Preview the mapping dataframe
    print("Gene mapping preview:")
    print(preview_df(mapping_df))
    
    # Apply the gene mapping to convert from probe-level to gene-level expression
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    # Preview the converted gene expression data
    print("\nGene expression data preview (after mapping):")
    print(f"Shape: {gene_data.shape}")
    print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")
    print(f"Sample columns: {gene_data.columns[:5].tolist()}")
    
    # Save the gene data to a CSV file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error in gene mapping: {e}")
    # In case of error, try to get the dataframe shape at least
    print(f"Gene expression dataframe shape: {gene_data.shape if 'gene_data' in locals() else 'Not available'}")
    print(f"Gene annotation dataframe shape: {gene_annotation.shape if 'gene_annotation' in locals() else 'Not available'}")


Gene mapping preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}

Gene expression data preview (after mapping):
Shape: (6421, 38)
First 5 gene symbols: ['A2BP1', 'A2M', 'AACS', 'AADACL1', 'AADAT']
Sample columns: ['GSM3580113', 'GSM3580114', 'GSM3580115', 'GSM3580117', 'GSM3580120']
Gene expression data saved to ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE125681.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print(f"Gene data shape before normalization: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains gene expression data for {trait} analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Gene data shape before normalization: (6421, 38)


Gene data shape after normalization: (6362, 38)


Normalized gene data saved to ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE125681.csv
Raw clinical data shape: (4, 39)
Clinical features:
                        GSM3580113  GSM3580114  GSM3580115  GSM3580117  \
Substance_Use_Disorder         1.0         1.0         1.0         1.0   
Age                           16.0        17.0        16.0        35.0   
Gender                         1.0         1.0         1.0         1.0   

                        GSM3580120  GSM3580121  GSM3580122  GSM3580124  \
Substance_Use_Disorder         1.0         1.0         1.0         1.0   
Age                           26.0        33.0        19.0        29.0   
Gender                         1.0         1.0         1.0         1.0   

                        GSM3580126  GSM3580128  ...  GSM3580154  GSM3580158  \
Substance_Use_Disorder         0.0         0.0  ...         1.0         0.0   
Age                           18.0        16.0  ...        30.0        34.0   
Gender          

Data shape after handling missing values: (38, 6365)
For the feature 'Substance_Use_Disorder', the least common label is '0.0' with 18 occurrences. This represents 47.37% of the dataset.
The distribution of the feature 'Substance_Use_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 22.25
  50% (Median): 29.0
  75%: 35.0
Min: 16.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 7 occurrences. This represents 18.42% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Substance_Use_Disorder/GSE125681.csv
